In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

from imblearn.pipeline import Pipeline

from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import ADASYN

from sklearn.preprocessing import LabelEncoder

from numpy.random import seed
seed(7)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [2]:
df = pd.read_csv("bearing_dataset_combined_0.5%_augmented_stage2.csv") 
df

,Unnamed: 0.1,Unnamed: 0,DE,FE,status,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,...,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0,0,0.014603,0.192920,0,NaN,NaN,NaN,NaN,NaN,...,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,NaN,NaN
1,1,1,0.054449,0.164360,0,0.014603,0.192920,NaN,NaN,NaN,...,0.081158,0.128719,0.034526,0.178640,0.014605,0.120639,0.054449,0.164275,0.057788,0.153113
2,2,2,0.107650,0.090811,0,0.054449,0.164360,NaN,NaN,NaN,...,0.119790,0.092518,0.120685,0.088653,0.014612,0.088902,0.107650,0.090642,0.100867,0.108119
3,3,3,0.133720,0.086496,0,0.107650,0.090811,NaN,NaN,NaN,...,0.122511,0.090060,0.120685,0.088653,0.014623,0.094407,0.133720,0.086243,0.121935,0.090760
4,4,4,0.112650,0.099235,0,0.133720,0.086496,NaN,NaN,NaN,...,0.099459,0.096357,0.097527,0.096153,0.014639,0.063512,0.112650,0.098897,0.110356,0.094509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,19995,0.028789,0.016642,1,0.081360,-0.055473,0.068246,-0.051100,-0.634921,...,-0.006899,0.044341,0.055074,-0.019415,0.077755,-0.031565,0.172571,0.171019,0.024043,0.012789
19996,19996,19996,-0.042766,0.073347,1,0.028789,0.016642,0.046730,-0.031669,-1.857739,...,-0.060277,0.078355,-0.060185,0.076326,0.048092,-0.031598,0.101041,0.227712,-0.033587,0.060660
19997,19997,19997,-0.077605,0.079305,1,-0.042766,0.073347,0.024378,-0.012973,-2.436305,...,-0.067836,0.060088,-0.060185,0.076326,-0.045850,-0.031621,0.066226,0.233658,-0.064827,0.069186
19998,19998,19998,-0.061333,0.044789,1,-0.077605,0.079305,0.008494,0.003727,-1.796750,...,-0.034584,0.005440,-0.034004,0.006574,-0.068749,-0.031635,0.082522,0.199130,-0.051737,0.034311


In [3]:
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0.1,Unnamed: 0,DE,FE,status,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,expanding_mean_DE,expanding_mean_FE,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0,0,0.014603,0.192920,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,NaN,NaN
1,1,1,0.054449,0.164360,0,0.014603,0.192920,NaN,NaN,NaN,NaN,0.034526,0.178640,0.081158,0.128719,0.034526,0.178640,0.014605,0.120639,0.054449,0.164275,0.057788,0.153113
2,2,2,0.107650,0.090811,0,0.054449,0.164360,NaN,NaN,NaN,NaN,0.058901,0.149364,0.119790,0.092518,0.120685,0.088653,0.014612,0.088902,0.107650,0.090642,0.100867,0.108119
3,3,3,0.133720,0.086496,0,0.107650,0.090811,NaN,NaN,NaN,NaN,0.077605,0.133647,0.122511,0.090060,0.120685,0.088653,0.014623,0.094407,0.133720,0.086243,0.121935,0.090760
4,4,4,0.112650,0.099235,0,0.133720,0.086496,NaN,NaN,NaN,NaN,0.084614,0.126764,0.099459,0.096357,0.097527,0.096153,0.014639,0.063512,0.112650,0.098897,0.110356,0.094509


In [3]:
df['status'].value_counts()[1]

100

In [4]:
df['status'].value_counts()[0]

19900

In [5]:
X = df.drop(['status'], axis=1)

y = df['status']

In [6]:
le = LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 0, ..., 1, 1, 1])

In [7]:
X=X.fillna(0)
X

,Unnamed: 0.1,Unnamed: 0,DE,FE,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,...,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0,0,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,0.000000,0.000000
1,1,1,0.054449,0.164360,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,...,0.081158,0.128719,0.034526,0.178640,0.014605,0.120639,0.054449,0.164275,0.057788,0.153113
2,2,2,0.107650,0.090811,0.054449,0.164360,0.000000,0.000000,0.000000,0.000000,...,0.119790,0.092518,0.120685,0.088653,0.014612,0.088902,0.107650,0.090642,0.100867,0.108119
3,3,3,0.133720,0.086496,0.107650,0.090811,0.000000,0.000000,0.000000,0.000000,...,0.122511,0.090060,0.120685,0.088653,0.014623,0.094407,0.133720,0.086243,0.121935,0.090760
4,4,4,0.112650,0.099235,0.133720,0.086496,0.000000,0.000000,0.000000,0.000000,...,0.099459,0.096357,0.097527,0.096153,0.014639,0.063512,0.112650,0.098897,0.110356,0.094509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,19995,0.028789,0.016642,0.081360,-0.055473,0.068246,-0.051100,-0.634921,-1.322713,...,-0.006899,0.044341,0.055074,-0.019415,0.077755,-0.031565,0.172571,0.171019,0.024043,0.012789
19996,19996,19996,-0.042766,0.073347,0.028789,0.016642,0.046730,-0.031669,-1.857739,-2.017084,...,-0.060277,0.078355,-0.060185,0.076326,0.048092,-0.031598,0.101041,0.227712,-0.033587,0.060660
19997,19997,19997,-0.077605,0.079305,-0.042766,0.073347,0.024378,-0.012973,-2.436305,-2.253240,...,-0.067836,0.060088,-0.060185,0.076326,-0.045850,-0.031621,0.066226,0.233658,-0.064827,0.069186
19998,19998,19998,-0.061333,0.044789,-0.077605,0.079305,0.008494,0.003727,-1.796750,-1.646885,...,-0.034584,0.005440,-0.034004,0.006574,-0.068749,-0.031635,0.082522,0.199130,-0.051737,0.034311


In [8]:
# Feature scaling

In [9]:
X=X.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
X

,DE,FE,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,expanding_mean_DE,expanding_mean_FE,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
0,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034047,0.176754,0.034526,0.178640,0.014603,0.192920,0.014603,0.192920,0.000000,0.000000
1,0.054449,0.164360,0.014603,0.192920,0.000000,0.000000,0.000000,0.000000,0.034526,0.178640,0.081158,0.128719,0.034526,0.178640,0.014605,0.120639,0.054449,0.164275,0.057788,0.153113
2,0.107650,0.090811,0.054449,0.164360,0.000000,0.000000,0.000000,0.000000,0.058901,0.149364,0.119790,0.092518,0.120685,0.088653,0.014612,0.088902,0.107650,0.090642,0.100867,0.108119
3,0.133720,0.086496,0.107650,0.090811,0.000000,0.000000,0.000000,0.000000,0.077605,0.133647,0.122511,0.090060,0.120685,0.088653,0.014623,0.094407,0.133720,0.086243,0.121935,0.090760
4,0.112650,0.099235,0.133720,0.086496,0.000000,0.000000,0.000000,0.000000,0.084614,0.126764,0.099459,0.096357,0.097527,0.096153,0.014639,0.063512,0.112650,0.098897,0.110356,0.094509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.028789,0.016642,0.081360,-0.055473,0.068246,-0.051100,-0.634921,-1.322713,0.013112,0.032767,-0.006899,0.044341,0.055074,-0.019415,0.077755,-0.031565,0.172571,0.171019,0.024043,0.012789
19996,-0.042766,0.073347,0.028789,0.016642,0.046730,-0.031669,-1.857739,-2.017084,0.013110,0.032769,-0.060277,0.078355,-0.060185,0.076326,0.048092,-0.031598,0.101041,0.227712,-0.033587,0.060660
19997,-0.077605,0.079305,-0.042766,0.073347,0.024378,-0.012973,-2.436305,-2.253240,0.013105,0.032772,-0.067836,0.060088,-0.060185,0.076326,-0.045850,-0.031621,0.066226,0.233658,-0.064827,0.069186
19998,-0.061333,0.044789,-0.077605,0.079305,0.008494,0.003727,-1.796750,-1.646885,0.013101,0.032772,-0.034584,0.005440,-0.034004,0.006574,-0.068749,-0.031635,0.082522,0.199130,-0.051737,0.034311


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((16000, 20), (4000, 20))

In [11]:
X_train

,DE,FE,lag_DE,lag_FE,rolling_mean_DE,rolling_mean_FE,DE_pct_change_5,FE_pct_change_5,expanding_mean_DE,expanding_mean_FE,cnv_DE,cnv_FE,pool_DE,pool_FE,tw_DE,tw_FE,drift_DE,drift_FE,trend_DE,trend_FE
17815,0.080943,-0.074991,0.060707,-0.058349,0.032127,-0.023803,1.984624,2.230143,0.012909,0.032717,0.071381,-0.056206,0.070825,-0.066670,-0.076041,-0.073188,0.194855,0.067281,0.071712,-0.062972
18370,0.043809,0.006986,0.053197,-0.045816,-0.012875,-0.019812,-1.763635,-2.888892,0.012949,0.032717,0.022324,0.038180,0.022322,0.037906,-0.034966,0.036458,0.161460,0.158402,0.035412,0.009246
1379,0.050485,-0.074785,0.069052,-0.125330,0.047475,-0.089667,-0.086790,-0.087721,0.012240,0.031837,0.027574,-0.054193,0.059769,-0.100057,0.006169,-0.036154,0.057692,-0.139762,0.044227,-0.076635
14763,-0.037134,0.116290,0.019401,0.053418,-0.000119,0.022659,0.092048,-48.166903,0.012642,0.032647,-0.047196,0.127384,-0.008866,0.084854,0.038590,0.021072,0.080301,0.183274,-0.028476,0.106272
7346,0.169810,0.141970,0.178780,0.065951,0.074177,0.080744,-12.628433,0.303793,0.012589,0.032363,0.155674,0.151785,0.153225,0.153580,0.004326,0.111691,0.263666,0.264774,0.163760,0.128770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,0.026703,-0.025065,0.001878,0.008013,-0.046462,0.044789,-1.359554,-1.580948,0.012406,0.032673,0.020766,-0.038013,0.014290,-0.008526,-0.035480,0.066172,0.146688,0.018325,0.018150,-0.022240
19648,0.081151,-0.162100,0.049442,-0.159230,0.035137,-0.084236,0.404337,1.405437,0.013091,0.032739,0.066743,-0.130239,0.067174,-0.130051,0.117397,-0.067158,0.217127,-0.004682,0.066235,-0.145358
9845,-0.027329,-0.034311,-0.095963,-0.007807,-0.052065,0.021338,-2.180212,-2.776575,0.012129,0.032408,-0.000142,-0.013487,-0.061646,-0.021059,-0.091142,-0.094397,0.078051,0.048954,-0.030249,-0.017669
10799,-0.021487,-0.029791,-0.023574,-0.064307,0.028043,-0.006076,-1.183933,-3.636372,0.012398,0.032583,-0.039515,0.013932,-0.022530,-0.047049,0.019424,-0.007195,0.088646,0.033273,-0.029206,-0.016385


In [12]:
# Feature scaling
# Fill missing values with a constant
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

X_train = X_train.replace([np.inf, -np.inf], 0) 
X_test = X_test.replace([np.inf, -np.inf], 0) 

cols = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])


In [13]:
scaler = StandardScaler()


# Feature scaling
# Fill missing values with a constant
X = X.fillna(0)

X = X.replace([np.inf, -np.inf], 0) 

cols = X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X, columns=[cols])



In [26]:
# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.99951


In [27]:
model = XGBClassifier()

clf_0 = model.fit(X_train, y_train)


In [28]:
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))

1.0


In [29]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_0)

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3983
     class 1       1.00      1.00      1.00        17

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



array([[3983,    0],
       [   0,   17]])

# Weighted XGBoost for Class Imbalance


In [30]:
# define model
#scale_pos_weight=total_negative_examples / total_positive_examples
#total number of examples in the majority class / total number of examples in the minority class.
#19900/100=199
model = XGBClassifier(scale_pos_weight=199)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.99973


In [31]:
clf_1 = model.fit(X_train, y_train)
pred_y_1 = clf_1.predict(X_test)

print(accuracy_score(pred_y_1, y_test))

0.99925


In [32]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_1, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_1)

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3983
     class 1       0.85      1.00      0.92        17

    accuracy                           1.00      4000
   macro avg       0.93      1.00      0.96      4000
weighted avg       1.00      1.00      1.00      4000



array([[3980,    3],
       [   0,   17]])

# Tune with GridSearch CV

In [33]:
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

In [35]:
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.999817 using {'scale_pos_weight': 25}
0.999514 (0.001232) with: {'scale_pos_weight': 1}
0.999596 (0.000902) with: {'scale_pos_weight': 10}
0.999817 (0.000511) with: {'scale_pos_weight': 25}
0.999640 (0.000949) with: {'scale_pos_weight': 50}
0.999729 (0.000758) with: {'scale_pos_weight': 75}
0.999628 (0.001252) with: {'scale_pos_weight': 99}
0.999670 (0.001042) with: {'scale_pos_weight': 100}
0.999760 (0.000965) with: {'scale_pos_weight': 1000}


In [39]:
# define model
model = XGBClassifier(scale_pos_weight=30)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % np.mean(scores))


Mean ROC AUC: 0.99975


In [40]:
clf_2 = model.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))



0.99975


In [41]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3983
     class 1       0.94      1.00      0.97        17

    accuracy                           1.00      4000
   macro avg       0.97      1.00      0.99      4000
weighted avg       1.00      1.00      1.00      4000



array([[3982,    1],
       [   0,   17]])

# aug+sampling

In [14]:
model = XGBClassifier(scale_pos_weight=30)
X_train=X_train.fillna(0)
# Define SMOTE-Tomek Links
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='all'))
# Define pipeline
pipeline=Pipeline(steps=[('r', resample), ('m', model)])

model = XGBClassifier()

clf_0 = pipeline.fit(X_train, y_train)
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))

target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_0)


#scale_pos_weight=199

0.99875
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3983
     class 1       0.77      1.00      0.87        17

    accuracy                           1.00      4000
   macro avg       0.89      1.00      0.94      4000
weighted avg       1.00      1.00      1.00      4000



array([[3978,    5],
       [   0,   17]])

In [15]:
# define model
model = XGBClassifier(scale_pos_weight=30)

X_train=X_train.fillna(0)

#Define SMOTE-ENN
resample=SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))
#Define pipeline
pipeline=Pipeline(steps=[('r', resample), ('m', model)])

clf_2 = pipeline.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))

target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

0.9985
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3983
     class 1       0.74      1.00      0.85        17

    accuracy                           1.00      4000
   macro avg       0.87      1.00      0.92      4000
weighted avg       1.00      1.00      1.00      4000



array([[3977,    6],
       [   0,   17]])

In [16]:
#ADASYN
ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X, y)
model = XGBClassifier(scale_pos_weight=30)

#Define ADASYN
resample=ADASYN(random_state=42)
clf_2 = pipeline.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))

pred_y_train = clf_2.predict(X_train)
print(accuracy_score(pred_y_train, y_train))


target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

0.9985
0.999625
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      3983
     class 1       0.74      1.00      0.85        17

    accuracy                           1.00      4000
   macro avg       0.87      1.00      0.92      4000
weighted avg       1.00      1.00      1.00      4000



array([[3977,    6],
       [   0,   17]])